In [9]:
import re
import json
import requests
from datetime import datetime

In [10]:
# ===============================
#  AMADEUS TOKEN
# ===============================
def obtener_token():
    url = "https://test.api.amadeus.com/v1/security/oauth2/token"

    data = {
        "grant_type": "client_credentials",
        "client_id": "fkkbxviKbgrQr7ppP00RoC6WgDTEAHGF",
        "client_secret": "MpUMWxQmAAu1ezAz"
    }

    r = requests.post(url, data=data)
    return r.json()["access_token"]

In [11]:
# ===============================
#  CIUDAD → IATA
# ===============================
def obtener_iata(ciudad):
    token = obtener_token()

    url = "https://test.api.amadeus.com/v1/reference-data/locations"
    params = {
        "keyword": ciudad,
        "subType": "CITY"
    }

    headers = {"Authorization": f"Bearer {token}"}

    r = requests.get(url, headers=headers, params=params)
    data = r.json()

    if data.get("data"):
        return data["data"][0]["iataCode"]

    return None

In [12]:
# ===============================
# FECHA INTELIGENTE
# ===============================
def normalizar_fecha(texto):
    hoy = datetime.now()
    año_actual = hoy.year

    meses = {
        "enero": "01", "febrero": "02", "marzo": "03", "abril": "04",
        "mayo": "05", "junio": "06", "julio": "07", "agosto": "08",
        "septiembre": "09", "octubre": "10", "noviembre": "11", "diciembre": "12"
    }

    # dd-mm-yyyy
    m = re.search(r"(\d{2})-(\d{2})-(\d{4})", texto)
    if m:
        return f"{m.group(1)}-{m.group(2)}-{m.group(3)}"

    # "25 de enero de 2019"
    m = re.search(r"(\d{1,2}) de (\w+) de (\d{4})", texto)
    if m and m.group(2) in meses:
        return f"{int(m.group(1)):02d}-{meses[m.group(2)]}-{m.group(3)}"

    # "25 de enero"
    m = re.search(r"(\d{1,2}) de (\w+)", texto)
    if m and m.group(2) in meses:
        return f"{int(m.group(1)):02d}-{meses[m.group(2)]}-{año_actual}"

    # solo mes
    for mes, num in meses.items():
        if mes in texto:
            return f"01-{num}-{año_actual}"

    # sin fecha
    return hoy.strftime("%d-%m-%Y")

In [22]:
# ===============================
# PROCESAR FRASE
# ===============================
def procesar_frase(frase):

    frase_lower = frase.lower()

    ciudades = ["quito","madrid", "barcelona", "roma", "frankfurt", "paris", "londres","sevilla"]
    origen = None
    destino = None

    for ciudad in ciudades:
        if ciudad in frase_lower:
            if not origen:
                origen = ciudad.capitalize()
            else:
                destino = ciudad.capitalize()

    fecha = normalizar_fecha(frase_lower)

    numeros = re.findall(r"\d+", frase_lower)
    cantidad = int(numeros[0]) if numeros else 1

    aerolineas = ["iberia", "lufthansa", "air france", "klm"]
    aerolinea = None
    for a in aerolineas:
        if a in frase_lower:
            aerolinea = a.capitalize()

    return origen, destino, fecha, cantidad, aerolinea

In [20]:
# ===============================
# GENERAR JSON FINAL
# ===============================
def generar_json_final(origen, destino, fecha, cantidad):
    return {
        "Origen": origen,
        "Ciudad Destino": destino,
        "Nombre Ciudad IATA From": obtener_iata(origen),
        "IATA To": obtener_iata(destino),
        "Fecha": fecha,
        "Pax": cantidad
    }

In [19]:
# ===============================
# BOT
# ===============================
def asistente():
    print("Hola, bienvenido a TravelBot. ¿Cómo te puedo ayudar?")

    while True:
        mensaje = input("Tú: ")

        if mensaje.lower() == "salir":
            print("Bot: Gracias por usar TravelBot. ¡Buen viaje!")
            break

        origen, destino, fecha, cantidad, aerolinea = procesar_frase(mensaje)

        if not origen or not destino:
            print("Bot: No pude detectar origen o destino.")
            continue

        print(f"Bot: Perfecto, Comienzo la búsqueda de tu viaje de {origen} a {destino} "
              f"para el {fecha} "
              f"con {aerolinea if aerolinea else 'aerolínea no especificada'}.")

        print("\nJSON enviado al sistema de reservas:")
        print(json.dumps(generar_json_final(origen, destino, fecha, cantidad),
                         indent=4, ensure_ascii=False))
        print("--------------------------------------------------")

In [23]:
asistente()

Hola, bienvenido a TravelBot. ¿Cómo te puedo ayudar?
Tú: Billete barato AirEuropa de Madrid a Sevilla
Bot: Perfecto, Comienzo la búsqueda de tu viaje de Madrid a Sevilla para el 05-02-2026 con aerolínea no especificada.

JSON enviado al sistema de reservas:
{
    "Origen": "Madrid",
    "Ciudad Destino": "Sevilla",
    "Nombre Ciudad IATA From": "MAD",
    "IATA To": "SVQ",
    "Fecha": "05-02-2026",
    "Pax": 1
}
--------------------------------------------------
Tú: Billete barato AirEuropa de Quito a Sevilla
Bot: Perfecto, Comienzo la búsqueda de tu viaje de Quito a Sevilla para el 05-02-2026 con aerolínea no especificada.

JSON enviado al sistema de reservas:
{
    "Origen": "Quito",
    "Ciudad Destino": "Sevilla",
    "Nombre Ciudad IATA From": null,
    "IATA To": "SVQ",
    "Fecha": "05-02-2026",
    "Pax": 1
}
--------------------------------------------------
Tú: salir
Bot: Gracias por usar TravelBot. ¡Buen viaje!
